# Week 05 Assignment: More Dynamic Programming

## String alignment

What is the best way to align the strings `CYCLE` and `BICYCLE`? Typographically, in left-to-right scripts, we tend to align to the left, like so:
```
CYCLE--
BICYCLE
```
The dashes (-) indicates a gap inserted to ensure the proper stretching for alignment.
Computationally, however, we may be interested in maximizing overlap of similar characters, and so the alignment 
```
--CYCLE
BICYCLE
```
may be more preferrable. For strings `CRANE` and `RAIN` we may have more than one alignment:
```
CRANE            CRA-NE
-RAIN            -RAIN-
```
The first option aligns as many letters as possible, while minimizing the use of spacing. The second option maximizes the number of aligned characters at the cost of additional spacing. Which of the two alignments is best? This depends on our priorities. And our priorities can be quantified in a way that determines the optimal alignment of two strings.

Let's assume that every time we use a space we pay a penalty $a_s<0$. Also everytime we have two characters that do not match, we also pay a penalty $a_\text{mismatch} <0$. And finally, when we have two characters matching, we pay no penalty $a_\text{match}\geq 0$. These penalties can be summarized as a function of two characters $x$ and $y$ stacked on top of eachother.

$$
a_{xy} =
\begin{cases}
  <0,\ \text{if}\ x \neq y\ \neq \text{space} \\
  <0,\ \text{if}\ x = \text{space} \neq y \\
  <0,\ \text{if}\ y = \text{space} \neq x  \\
  \geq 0,\ \text{if}\ x = y
\end{cases}
$$

There is no penalty for a space stacked over a space.

A simple penalty function that favors alignments even at the cost of extra spaces, may look like:

$$
a_{xy} =
\begin{cases}
  -5,\ \text{if}\ x \neq y\ \neq \text{space} \\
  -1,\ \text{if}\ (x = \text{space} \neq y)\ \text{or}\ (y = \text{space} \neq x )\\
  1,\ \text{if}\ x = y
\end{cases}
$$

For small enough strings, we can find the optimal alignment relatively easy, even with brute force. For two strings with $m$ and $n$ characters respectively, there are 

$$
{\binom{m+n}{n}} = \dfrac{(m+n)!}{m!n!}
$$

possible alignments. If $m=100$ and $m=5$, that's 

$$
\binom{105}{5}=\dfrac{105!}{5!(105-5)!}=96,560,646
$$

and even a laptop can go through them in a few seconds. When the strings get longer, for example $m=10,000$ and $n=500$, the number of alignments becomes astronomical, approximately $1.8\times 10^{871}$ and there is no time to compute so many cases.

## Optimal substructure in string alignment

Consider two strings

\begin{align*}
X_m & = x_1 x_2 \ldots x_m \\
Y_n & = y_1 y_2 \ldots y_n
\end{align*}
whose lengths $m,n$ are not necessarily equal. Their optimal alignment is a pair of strings $\bar{X}$ and $\bar{Y}$ with equal length $L\geq\max{(m,n)}$ such that

$$
{\arg\min} \sum_{\begin{array}{c}\bar x_i\in\bar X\\ \bar y_j\in\bar Y\end{array}} a_{\bar x_i\ \bar y_j}
$$

The strings we are interested in aligning typically represent genetic codes. Their length could be large enough to make brute force computation intractable. Finding the optimal alignment $(\bar X, \bar Y)$ is impossible in these cases.

Instead imagine that the optimal alignment $(\bar X, \bar Y)$ is given to us. What would it look like? Both its strings have the same length $L\geq\max{(m,n)}$. We can further split the optimally aligned strings into two parts:

\begin{align*}
\bar X & = X' | \bar x_L \\
\bar Y & = Y' | \bar y_L
\end{align*}

where $X', Y'$ are the first $L-1$ characters of the aligned strings and $\bar x_L$, $\bar y_L$ are the last characters of the aligned strings. Focusing on the last character of $(\bar X, \bar Y)$, we expect one of the following three cases, with respect to the contents of input strings $X_m$ and $Y_n$:

$$
\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=
\begin{cases}
\left|\begin{array}{c}x_m\\ y_n\end{array}\right|: \text{the last character of}\ X_m\ \text{over the last character of}\ Y_n \\ \\
\left|\begin{array}{c}x_m\\ -\end{array}\right|: \text{the last character of}\ X_m\ \text{over a space} \\ \\
\left|\begin{array}{c}-\\ y_n\end{array}\right|: \text{a space over the last character of}\ Y_n
\end{cases}
$$

Each of the cases above reveals important properties for $X'$ and $Y'$, the immediately shorter substrings of the alignment $\bar X$, $\bar Y$.

### Case for $\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=\left|\begin{array}{c}x_m\\ y_n\end{array}\right|$

When the last column of the optimal alignment $(\bar X, \bar Y)$ contains the last characters of the input strings $X_m$ and $Y_n$, then we know that the strings $X'$ and $Y'$ are an optimal alignment of the substrings $X_{m-1}$ and $Y_{n-1}$.

### Case for $\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=\left|\begin{array}{c}x_m\\ -\end{array}\right|$

When the last column of the optimal alignment $(\bar X, \bar Y)$ contains the last character of the input string $X_m$ and a space, then we know that the strings $X'$ and $Y'$ are an optimal alignment of the substrings $X_{m-1}$ and $Y_{n}$.

### Case for $\left|\begin{array}{c}\bar x_L\\ \bar y_L\end{array}\right|=\left|\begin{array}{c}-\\ y_n\end{array}\right|$

When the last column of the optimal alignment $(\bar X, \bar Y)$ contains a space and the last character of the input string $Y_n$, then we know that the strings $X'$ and $Y'$ are an optimal alignment of the substrings $X_{m}$ and $Y_{n-1}$.

## Computing penalties

The total penalty for aligning strings $X_m$ and $Y_n$ is measured by some function $P(m,n)$. Based on the analysis above, the alignment of the strings can result from one of the following three cases:
* The alignment of $X_{m-1}$ and $Y_{n-1}$ plus a the penalty associated with $x_m$ aligned with $y_n$; or,
* The alignment of $X_{m-1}$ and $Y_{n}$ plus a the penalty associated with $x_m$ aligned with a space; or,
* The alignment of $X_{m}$ and $Y_{n-1}$ plus a the penalty associated with a space aligned with $y_n$.

Which of these cases prevails depends on the minimum cost associated with it. We can write:

\begin{align*}
P(m,n) = 
    \min( & P(m-1, n-1) + a_{x_m, y_n}, \\
           & P(m-1, n) + a_\text{space}, \\
           & P(m, n-1)+ a_\text{space} )
\end{align*}

# Your assignment



### Reading


* [Dynamic Programming](https://jeffe.cs.illinois.edu/teaching/algorithms/book/03-dynprog.pdf) (Chapter 3 from Jeff Erickson's book).
* [Dynamic Programming](https://web.mit.edu/15.053/www/AMP-Chapter-11.pdf) (Chapter 11, from Applied Mathematical Programming, by Bradley, Hax, and Magnanti).
* [The Theory of Dynamic Programming](https://www.rand.org/content/dam/rand/pubs/papers/2008/P550.pdf) Richard Bellman's 1954 paper summarizing his work.
* [Dynamic Programming](https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf) (Chapter 6 from Algorithms by Dasgupta, Papadimitriou, and Vazirani).'


# Coding requirements

- You may _not_ import modules in your code without explicit permission from Leo. Basically this means no `import` or `include` or similar statements in your programs.

- You may _not_ use statements like `break` to end loops or `continue` and `pass` to move through branching.

- When possible, methods that return values should have only one return statement. This is no longer a strict requirement (if you took COMP 271/272 with me, you know what I am talking about). In general, there is no good reason for a method with 20-25 lines of code at most to have multiple return statements.

- Your code should be neat and well documented. If you are coding with Visual Studio Code, there are extensions that can do a great job formatting your program. For Python, consider installing the **Black Formatter** by Microsoft.

- If you code in Python, learn to use type hints. They are annoying but useful.

- Use a standard style guide for your code. I like Google's style guides for [Java](https://google.github.io/styleguide/javaguide.html) and [Python](https://google.github.io/styleguide/pyguide.html).

- If you are using Jupyter notebooks, spend some time exploring MarkDown syntax for documentation and LaTeX for mathmetical typesetting. Good skills to have.

# Finals week policy

There is no final exam for the course. There will be a final assignemnt that will be published the week before finals and will be due the week of finals. Additionally, 8 students in the course will be invited randomly to a brief meeting with the instructor during the course's final exam slot. If you are selected for a brief meeting, we'll spend about 15 minutes during the final exam slot to review your work. This interview will cover coding practices based on your past assignments. It is meant as a checkpoint to ensure that you have internalized the work you submitted.
